# Тесты эстиматоров взаимной информации

## Преамбула

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns

In [ ]:
font = {'family' : 'DejaVu Sans',
        'size'   : 18}

matplotlib.rc('font', **font)

#### Импортирование модуля

In [ ]:
import sys
sys.path.insert(0, './py')

In [ ]:
import mutinfo.estimators.mutual_information as mi_estimators
from mutinfo.utils.dependent_norm import multivariate_normal_from_MI

In [ ]:
global_n_jobs = 32

#### Стандартные тесты с произвольным преобразованием

In [ ]:
def perform_normal_test(mi, n_samples, X_dimension, Y_dimension, X_map = None, Y_map = None, verbose = 0):
    # Генерация.
    random_variable = multivariate_normal_from_MI(X_dimension, Y_dimension, mi)
    X_Y = random_variable.rvs(n_samples)
    X = X_Y[:, 0:X_dimension]
    Y = X_Y[:, X_dimension:X_dimension + Y_dimension]
        
    # Применение преобразования.
    if not X_map is None:
        X = X_map(X)
           
    if not Y_map is None:
        Y = Y_map(Y)
            
    if (not X_map is None) or (not Y_map is None):
        X_Y = np.concatenate([X, Y], axis=1)

    # Оценка энтропии.
    mi_estimator = mi_estimators.MutualInfoEstimator(Y_is_discrette = False, n_jobs = global_n_jobs)
    mi_estimator.fit(X, Y, verbose = verbose)
    
    return mi_estimator.predict(X, Y, verbose = verbose)

#### График оценки взаимной информации

## Зависимость оценки от истинного значения

In [ ]:
def perform_normal_test_MI(MI, n_samples, X_dimension, Y_dimension, X_map = None, Y_map = None, verbose = 0):
    """
    Вычислить оценки взаимной информации для разных истинных значений.
    """
    n_exps = len(MI)
    
    # Оценки взаимной информации.
    estimated_MI = []

    # Проведение тестов.
    for n_exp in range(n_exps):
        print("\nn_exp = %d/%d\n------------\n" % (n_exp + 1, n_exps))
        estimated_MI.append(perform_normal_test(MI[n_exp], n_samples, X_dimension, Y_dimension,
                                                X_map, Y_map, verbose))
        
    return estimated_MI

In [ ]:
def plot_estimated_MI(MI, estimated_MI, title):
    estimated_MI_mean = np.array([estimated_MI[index][0] for index in range(len(estimated_MI))])
    estimated_MI_std  = np.array([estimated_MI[index][1] for index in range(len(estimated_MI))])
    
    fig_normal, ax_normal = plt.subplots()

    fig_normal.set_figheight(11)
    fig_normal.set_figwidth(16)

    # Сетка.
    ax_normal.grid(color='#000000', alpha=0.15, linestyle='-', linewidth=1, which='major')
    ax_normal.grid(color='#000000', alpha=0.1, linestyle='-', linewidth=0.5, which='minor')

    ax_normal.set_title(title)
    ax_normal.set_xlabel("$I(X,Y)$")
    ax_normal.set_ylabel("$\\hat I(X,Y)$")
    
    ax_normal.minorticks_on()
    
    #ax_normal.set_yscale('log')
    #ax_normal.set_xscale('log')

    ax_normal.plot(MI, MI, label="$I(X,Y)$")
    ax_normal.plot(MI, estimated_MI_mean, label="$\\hat I(X,Y)$")
    ax_normal.fill_between(MI, estimated_MI_mean + estimated_MI_std, estimated_MI_mean - estimated_MI_std, alpha=0.2)

    ax_normal.legend(loc='upper left')

    ax_normal.set_xlim((0.0, None))
    ax_normal.set_ylim((0.0, None))

    plt.show();

### Глобальные параметры тестов

In [ ]:
# Исследуемые значения взаимной информации.
#MI = [0.0, 0.1, 0.2, 0.3, 0.5, 0.7, 1.0, 1.5, 2.0, 3.0, 5.0, 6.0, 8.0, 10.0]
MI = [0.0, 0.1, 0.2, 0.5, 1.0, 2.0, 5.0, 10.0]
#MI = [0.0, 2.0, 5.0]
n_exps = len(MI)

# Число экземпляров и размерности векторов X и Y.
n_samples = 10000
X_dimension = 4
Y_dimension = 4

### Нормальный случайный вектор

In [ ]:
# Оценки взаимной информации.
estimated_MI = perform_normal_test_MI(MI, n_samples, X_dimension, Y_dimension, verbose=10)

In [ ]:
plot_estimated_MI(MI, estimated_MI, "Нормальные векторы")

### Равномерные распределения

Применим к компонентам нормального случайного вектора их функцию распределения.

In [ ]:
from scipy.special import ndtr

In [ ]:
# Оценки взаимной информации.
estimated_MI = perform_normal_test_MI(MI, n_samples, X_dimension, Y_dimension, X_map = ndtr, Y_map = ndtr,
                                      verbose=10)

In [ ]:
plot_estimated_MI(MI, estimated_MI, "Равномерное распределение")

### Полиномы

Получим равномерные распределения согласно второму пункту. Далее применим следующее преобразование:

$$
\begin{cases}
x'_1 = (x_1 + 1)^2 \\
x'_2 = x_1 + (x_2 + 1)^2 \\
\ldots \\
x'_d = x_{d-1} + (x_d + 1)^2
\end{cases}
$$

In [ ]:
def poly_mapping(X):
    """
    Преобразование нормального вектора согласно формуле выше.
    """
    
    assert len(X.shape) == 2
    
    X = ndtr(X)
    new_X = np.zeros_like(X)
    for index in range(X.shape[0]):
        new_X[index][0] = (X[index][0] + 1)**2
        for jndex in range(1, X.shape[1]):
            new_X[index][jndex] = X[index][jndex - 1] + (X[index][jndex] + 1.0)**2
            
    return new_X

In [ ]:
# Оценки взаимной информации.
estimated_MI = perform_normal_test_MI(MI, n_samples, X_dimension, Y_dimension, X_map = poly_mapping,
                                      Y_map = poly_mapping, verbose=10)

In [ ]:
plot_estimated_MI(MI, estimated_MI, "Полиномы")

### Кольца

Получим равномерные распределения согласно второму пункту. Далее применим следующее преобразование:

$$
\begin{cases}
x' = [R \cdot x + r \cdot (1 - x)] \cdot \cos(2 \pi y) \\
y' = [R \cdot x + r \cdot (1 - x)] \cdot \sin(2 \pi y) \\
\end{cases}
$$

Здесь требуется размерность $ 2 $ у обоих векторов.

In [ ]:
r = 1.0
R = 2.0

def ring_mapping(X):
    """
    Нормальный вектор в кольцо.
    """
    
    assert len(X.shape) == 2
    assert X.shape[1] == 2
    
    X = ndtr(X)
    new_X = np.zeros_like(X)
    for index in range(X.shape[0]):
        rho = R * X[index][0] + r * (1.0 - X[index][0])
        phi = 2.0 * np.pi * X[index][1]
        
        new_X[index][0] = rho * np.cos(phi)
        new_X[index][1] = rho * np.sin(phi)
    
    return new_X

In [ ]:
def _rings_pp():
    _X_Y = multivariate_normal_from_MI(2, 2, mutual_information=10.0).rvs(10000)
    _X = _X_Y[:, 0:2]
    _Y = _X_Y[:, 2:4]
    _X = ring_mapping(_X)
    _Y = ring_mapping(_Y)
    _X_Y = np.concatenate([_X, _Y], axis=1)

    pp = sns.pairplot(pd.DataFrame(_X_Y), height = 2.0, aspect=1.6,
                      plot_kws=dict(edgecolor="k", linewidth=0.0, alpha=0.05, size=0.01, s=0.01),
                      diag_kind="kde", diag_kws=dict(shade=True))

    fig = pp.fig
    fig.subplots_adjust(top=0.93, wspace=0.3)
    t = fig.suptitle("Попарные графики", fontsize=14)
    
_rings_pp()

In [ ]:
# Оценки взаимной информации.
estimated_MI = perform_normal_test_MI(MI, n_samples, 2, 2, X_map = ring_mapping, Y_map = ring_mapping, verbose=10)

In [ ]:
plot_estimated_MI(MI, estimated_MI, "Кольца")

## Зависимость оценки от размерности

In [ ]:
def perform_normal_test_dim(mi, n_samples, dimensions, X_map = None, Y_map = None, verbose = 0):
    """
    Вычислить оценки взаимной информации для разных истинных значений.
    """
    n_exps = len(dimensions)
    
    # Оценки взаимной информации.
    estimated_MI = []

    # Проведение тестов.
    for n_exp in range(n_exps):
        print("\nn_exp = %d/%d\n------------\n" % (n_exp + 1, n_exps))
        estimated_MI.append(perform_normal_test(mi, n_samples, dimensions[n_exp], dimensions[n_exp],
                                                X_map, Y_map, verbose))
        
    return estimated_MI

In [ ]:
def plot_estimated_dim(dimensions, mi, estimated_MI, title):
    estimated_MI_mean = np.array([estimated_MI[index][0] for index in range(len(estimated_MI))])
    estimated_MI_std  = np.array([estimated_MI[index][1] for index in range(len(estimated_MI))])
    
    fig_normal, ax_normal = plt.subplots()

    fig_normal.set_figheight(11)
    fig_normal.set_figwidth(16)

    # Сетка.
    ax_normal.grid(color='#000000', alpha=0.15, linestyle='-', linewidth=1, which='major')
    ax_normal.grid(color='#000000', alpha=0.1, linestyle='-', linewidth=0.5, which='minor')

    ax_normal.set_title(title)
    ax_normal.set_xlabel("Размерность $ X $ и $ Y $")
    ax_normal.set_ylabel("$\\hat I(X,Y)$")
    
    ax_normal.minorticks_on()
    
    #ax_normal.set_yscale('log')
    #ax_normal.set_xscale('log')

    ax_normal.plot(dimensions, np.ones_like(dimensions) * mi, label="$I(X,Y)$")
    ax_normal.plot(dimensions, estimated_MI_mean, label="$\\hat I(X,Y)$")
    ax_normal.fill_between(dimensions, estimated_MI_mean + estimated_MI_std, estimated_MI_mean - estimated_MI_std, alpha=0.2)

    ax_normal.legend(loc='upper left')

    ax_normal.set_xlim((0.0, None))
    ax_normal.set_ylim((0.0, None))

    plt.show();

### Глобальные параметры тестов

In [ ]:
#dimensions = [1, 2, 3, 4, 5, 6, 8, 10, 12, 14, 16, 20, 25, 30, 40]
dimensions = [1, 2, 4, 6, 8, 12, 16, 20, 30, 40]
mi = 2.0

### Нормальный случайный вектор

In [ ]:
# Оценки взаимной информации.
estimated_MI = perform_normal_test_dim(mi, n_samples, dimensions, verbose=10)

In [ ]:
plot_estimated_dim(dimensions, mi, estimated_MI, "Нормальные векторы")